## **📋 模块 1：环境初始化与依赖导入**

In [1]:
import sys
import os
import gc
import time
import psutil
import numpy as np
import pandas as pd
import polars as pl
from typing import Tuple, Dict, Any

from mars.feature.binning import MarsNativeBinner, MarsOptimalBinner
from mars.utils.logger import set_log_level, logger

# 设置日志级别，避免刷屏
set_log_level("INFO")

def get_memory_usage() -> float:
    """获取当前进程内存占用 (MB)"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

print("✅ 环境初始化完成。Mars 库加载成功。")


    __________________________________________________________________________
       __  ___ ___    ____  _____
      /  |/  //   |  / __ \/ ___/
     / /|_/ // /| | / /_/ /\__ \ 
    / /  / // ___ |/ _, _/___/ / 
   /_/  /_//_/  |_/_/ |_|/____/  
                                 
    MODELING ANALYSIS RISK SCORE 
    __________________________________________________________________________
    Version: 0.0.1 | Copyright (c) 2026 Christian Li
    High-performance Risk Modeling Toolkit powered by Polars
    __________________________________________________________________________

✅ 环境初始化完成。Mars 库加载成功。


## **🛠️ 模块 2：逻辑正确性测试 (Unit Tests)**
- 验证核心功能、边界条件（空值、单一值）和特殊逻辑（低基数、空箱合并）是否正确。

In [2]:
# ==========================================
# 模块 2: 逻辑正确性测试 (Logic Verification) - 适配新协议版
# ==========================================

def run_logic_tests():
    print("\n" + "="*60)
    print("🧪 开始逻辑正确性测试 (Unit Tests) - 适配新索引协议")
    print("="*60)
    
    # ------------------------------------------------------
    # [Case 1] 低基数测试 (Low Cardinality)
    # ------------------------------------------------------
    print("\n[Case 1] 低基数测试 (Low Cardinality)")
    df = pl.DataFrame({"feature": [1]*50 + [2]*30 + [3]*20})
    binner = MarsNativeBinner(method="uniform", n_bins=10)
    binner.fit(df, None)
    cuts = binner.bin_cuts_["feature"]
    print(f"   -> Result Cuts: {cuts}")
    assert len(cuts) <= 5, "❌ 失败: 切点没有自动缩减"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 2] 空箱合并测试 (Empty Bin Pruning)
    # ------------------------------------------------------
    print("\n[Case 2] 空箱合并测试 (Empty Bin Pruning)")
    df = pl.DataFrame({"feature": [0.0]*100 + [100.0]*100})
    binner = MarsNativeBinner(method="uniform", n_bins=5, remove_empty_bins=True)
    binner.fit(df, None)
    cuts = binner.bin_cuts_["feature"]
    print(f"   -> Result Cuts: {cuts}")
    assert len(cuts) < 7, "❌ 失败: 空箱未被移除"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 3] 边界值测试 (Nulls & Constant)
    # ------------------------------------------------------
    print("\n[Case 3] 边界值测试 (Nulls & Constant)")
    df_edge = pl.DataFrame({
        "all_null": [None, None, np.nan],
        "constant": [5.0, 5.0, 5.0]
    })
    binner = MarsNativeBinner(method="quantile", n_bins=5)
    binner.fit(df_edge, None)
    
    # 默认 index 模式测试
    res = binner.transform(df_edge, return_type="index")
    null_res = res["all_null_bin"].to_list()
    # 验证索引是否为 -1 (Missing)
    assert all(x == -1 for x in null_res), f"❌ 全空列处理失败: {null_res}"
    
    # Label 模式测试
    res_label = binner.transform(df_edge, return_type="label")
    assert all(x == "Missing" for x in res_label["all_null_bin"].to_list()), "❌ Label模式Missing映射错误"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 4] 多特殊值独立隔离测试
    # ------------------------------------------------------
    print("\n[Case 4] 多特殊值独立隔离 (Multi Special Values)")
    print("场景: 混合 -999, -998。期望: -999->-3, -998->-4")
    df_spec = pl.DataFrame({"feature": [10, 20, 30]*10 + [-999]*5 + [-998]*5})
    binner = MarsNativeBinner(method="quantile", n_bins=3, special_values=[-999, -998])
    binner.fit(df_spec, None)
    
    res = binner.transform(df_spec, return_type="index")
    unique_indices = res["feature_bin"].unique().to_list()
    
    # 验证协议索引
    assert -3 in unique_indices, f"❌ 失败: 特殊值-999未映射到-3, 得到: {unique_indices}"
    assert -4 in unique_indices, f"❌ 失败: 特殊值-998未映射到-4, 得到: {unique_indices}"
    
    # 验证 Label 模式
    res_label = binner.transform(df_spec, return_type="label")
    unique_labels = res_label["feature_bin"].unique().to_list()
    assert "Special_-999" in unique_labels, "❌ Label模式特殊值1映射错误"
    assert "Special_-998" in unique_labels, "❌ Label模式特殊值2映射错误"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 5] CART 决策树有效性
    # ------------------------------------------------------
    print("\n[Case 5] 决策树有效性 (CART Logic)")
    X_cart = np.random.rand(1000, 1)
    y_cart = (X_cart[:, 0] > 0.5).astype(int)
    df_cart = pl.DataFrame(X_cart, schema=["feature"])
    binner = MarsNativeBinner(method="cart", n_bins=3, n_jobs=1)
    binner.fit(df_cart, y_cart)
    cuts = binner.bin_cuts_["feature"]
    print(f"   -> DT Cuts: {cuts}")
    has_cut_near_05 = any(0.45 < c < 0.55 for c in cuts if c not in [float('-inf'), float('inf')])
    assert has_cut_near_05, "❌ 失败: CART 未能识别明显的切点 (0.5)"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 6] Int 类型 + Null 转换稳定性
    # ------------------------------------------------------
    print("\n[Case 6] Int 类型含空值测试 (Int Null Safety)")
    df_int = pl.DataFrame({"feature": [1, 2, None, 4]}, schema={"feature": pl.Int64})
    binner = MarsNativeBinner(method="quantile", n_bins=3)
    
    try:
        binner.fit(df_int, None) 
        res = binner.transform(df_int, return_type="index")
        
        # 验证 Missing 是否被正确标记为 -1
        null_rows = res.filter(pl.col("feature").is_null())
        null_idx = null_rows["feature_bin"][0]
        
        assert null_idx == -1, f"Int Null 标记错误: {null_idx}"
        print("   ✅ 通过 (Fit & Transform 均正常)")
        
    except Exception as e:
        print(f"   ❌ 失败: 发生崩溃 - {e}")
        raise e

    # ------------------------------------------------------
    # [Case 7] 极度重读去重 (Heavy Duplicates)
    # ------------------------------------------------------
    print("\n[Case 7] 极度重读去重 (Heavy Duplicates)")
    df_dup = pl.DataFrame({"feature": [0.0]*900 + [1.0]*50 + [2.0]*50})
    binner = MarsNativeBinner(method="quantile", n_bins=10)
    binner.fit(df_dup, None)
    cuts = binner.bin_cuts_["feature"]
    print(f"   -> Cleaned Cuts: {cuts}")
    assert len(cuts) == len(set(cuts)), "❌ 失败: 切点包含重复值"
    assert len(cuts) < 6, "❌ 失败: 切点未充分合并"
    print("   ✅ 通过")
    
    # ------------------------------------------------------
    # [Case 8] 混合类型抗毁性测试 (Type Safety Crash Test)
    # ------------------------------------------------------
    print("\n[Case 8] 混合类型配置抗毁性 (Type Safety)")
    print("场景: 全局配置包含字符串('unknown')，但当前列是 Int64。期望: 不崩溃，且能识别数值型缺失(-999)。")
    
    df_safe = pl.DataFrame(
        {"age": [25, 30, -999, 40]}, 
        schema={"age": pl.Int64}
    )
    
    # 故意混入字符串，测试 Binner 是否会自动过滤
    binner = MarsNativeBinner(
        method="quantile", 
        n_bins=3, 
        missing_values=[-999, "unknown", "N/A"] # "unknown" 对 Int64 是致命的，除非内部已过滤
    )
    
    try:
        binner.fit(df_safe, None)
        res = binner.transform(df_safe, return_type="index")
        
        # 验证 -999 是否被正确识别为 Missing (-1)
        missing_idx = res.filter(pl.col("age") == -999)["age_bin"][0]
        assert missing_idx == -1, f"❌ 失败: 数值型缺失值未被识别 (Got {missing_idx})"
        print("   ✅ 通过 (类型安全过滤生效，未发生 Schema Error)")
    except Exception as e:
        print(f"   ❌ 失败: 触发了类型错误 - {e}")
        raise e

    # ------------------------------------------------------
    # [Case 9] 类别特征分箱 (Categorical Binning)
    # ------------------------------------------------------
    print("\n[Case 9] 类别特征分箱 (Categorical Logic)")
    # 构造数据: A(高频), B(中频), C(低频), None(缺失)
    df_cat = pl.DataFrame({"city": ["A"]*60 + ["B"]*30 + ["C"]*10 + [None]*5})
    
    # 模拟 fit 过程 (因为 NativeBinner 目前主要针对数值，类别通常需要预处理或手动指定 cat_cuts_)
    # 这里我们手动注入 cat_cuts_ 来模拟类别分箱已 Fit 的状态
    binner_cat = MarsNativeBinner()
    binner_cat.cat_cuts_ = {"city": [["A"], ["B"], ["C"]]} # 3个箱
    binner_cat.missing_values = ["unknown"] # 测试配置
    
    res = binner_cat.transform(df_cat, return_type="index")
    res_label = binner_cat.transform(df_cat, return_type="label")
    
    # 验证 Null -> -1
    null_idx = res.filter(pl.col("city").is_null())["city_bin"][0]
    assert null_idx == -1, "❌ 类别型 Null 未映射到 -1"
    
    # 验证 Label 映射
    label_a = res_label.filter(pl.col("city") == "A")["city_bin"][0]
    assert "A" in label_a, "❌ 类别型 Label 映射错误"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 10] WOE 映射与单调性 (WOE Calculation)
    # ------------------------------------------------------
    print("\n[Case 10] WOE 计算验证 (WOE Materialization)")
    # 构造强相关数据: x越大，y=1概率越大
    X_woe = pl.DataFrame({"score": np.linspace(0, 100, 1000)})
    # y 在 score < 50 时全0，> 50 时全1 (WOE 应该单调递增)
    y_woe = (X_woe["score"] > 50).cast(pl.Int8).to_numpy()
    
    binner_woe = MarsNativeBinner(n_bins=5, method="quantile")
    binner_woe.fit(X_woe, y_woe)
    
    # 计算 Stats (包含 WOE)
    stats = binner_woe.compute_bin_stats(X_woe, y_woe)
    
    # 提取 score 的 WOE 列表
    woe_list = stats.filter(
        (pl.col("feature")=="score_bin") & (pl.col("bin_index") >= 0) # 只看正常箱
    ).sort("bin_index")["woe"].to_list()
    
    print(f"   -> WOE Trend: {[round(x, 2) for x in woe_list]}")
    
    # 简单验证单调性 (允许轻微波动，但这里是强规则)
    is_monotonic = all(x <= y for x, y in zip(woe_list, woe_list[1:]))
    assert is_monotonic, "❌ 失败: WOE 不具备单调性"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 11] Lazy 模式全链路测试 (Lazy Execution)
    # ------------------------------------------------------
    print("\n[Case 11] Lazy 模式全链路 (Lazy Execution)")
    df_lazy = pl.DataFrame({"val": range(100)}).lazy()
    
    binner_lazy = MarsNativeBinner(n_bins=5)
    # Fit 必须用 Eager (为了获取切点)，但 transform 可以用 Lazy
    binner_lazy.fit(df_lazy.collect(), None)
    
    # 测试 Lazy Transform
    lf_res = binner_lazy.transform(df_lazy, return_type="index", lazy=True)
    
    assert isinstance(lf_res, pl.LazyFrame), "❌ 返回类型不是 LazyFrame"
    
    # 真正执行
    df_res = lf_res.collect()
    assert "val_bin" in df_res.columns, "❌ Lazy 转换未生成目标列"
    print("   ✅ 通过")

    # ------------------------------------------------------
    # [Case 12] 特殊值与类别兜底共存 (Special vs Other)
    # ------------------------------------------------------
    print("\n[Case 12] 特殊值(-3) 与 类别兜底(-2) 区分")
    # 模拟：类别特征，有 'A', 'B'，特殊值 'S'，未知值 'Z'(Other)
    binner_mix = MarsNativeBinner(special_values=["S"])
    binner_mix.cat_cuts_ = {"code": [["A"], ["B"]]} # 仅定义了 A 和 B
    
    df_mix = pl.DataFrame({"code": ["A", "B", "S", "Z"]})
    
    res = binner_mix.transform(df_mix, return_type="index")
    
    # 验证:
    # A/B -> >=0
    # S -> -3 (特殊值)
    # Z -> -2 (Other, 因为不在 cat_cuts_ 里且不是特殊值)
    
    idx_s = res.filter(pl.col("code")=="S")["code_bin"][0]
    idx_z = res.filter(pl.col("code")=="Z")["code_bin"][0]
    
    assert idx_s == -3, f"❌ 特殊值 S 索引错误: {idx_s} (Expect -3)"
    assert idx_z == -2, f"❌ 未知值 Z 索引错误: {idx_z} (Expect -2)"
    print("   ✅ 通过 (协议 -2/-3 隔离成功)")

    print("\n🎉🎉🎉 逻辑测试全部通过！适配新索引协议成功！🎉🎉🎉")

run_logic_tests()


🧪 开始逻辑正确性测试 (Unit Tests) - 适配新索引协议

[Case 1] 低基数测试 (Low Cardinality)
[MARS] 2026-01-15 23:54:50 - INFO - ⚙️ Fitting bins for 1 features (Native Mode: uniform)...
[MARS] 2026-01-15 23:54:50 - INFO - ⏱️ [MarsNativeBinner._fit_impl] finished in 0.0034s
   -> Result Cuts: [-inf, 1.5, 2.5, inf]
   ✅ 通过

[Case 2] 空箱合并测试 (Empty Bin Pruning)
[MARS] 2026-01-15 23:54:50 - INFO - ⚙️ Fitting bins for 1 features (Native Mode: uniform)...
[MARS] 2026-01-15 23:54:50 - INFO - ⏱️ [MarsNativeBinner._fit_impl] finished in 0.0010s
   -> Result Cuts: [-inf, 50.0, inf]
   ✅ 通过

[Case 3] 边界值测试 (Nulls & Constant)
[MARS] 2026-01-15 23:54:50 - WARNING - Feature 'constant' is constant. Skipped.
[MARS] 2026-01-15 23:54:50 - INFO - ⚙️ Fitting bins for 1 features (Native Mode: quantile)...
[MARS] 2026-01-15 23:54:50 - INFO - ⏱️ [MarsNativeBinner._fit_impl] finished in 0.0011s
   ✅ 通过

[Case 4] 多特殊值独立隔离 (Multi Special Values)
场景: 混合 -999, -998。期望: -999->-3, -998->-4
[MARS] 2026-01-15 23:54:50 - INFO - ⚙️ Fitting bi

## **🚀 模块 3：大规模压力测 (Stress Test)**
- 说明：使用 20万行 x 2000列 的大宽表，验证内存稳定性及三种模式（Quantile/Uniform/CART）的绝对性能。

In [ ]:
# --- 配置参数 ---
N_ROWS = 100_000  # 100万行 (可根据机器配置调整)
N_COLS = 5000         # 50列
MISSING_VAL = -999.0
SPECIAL_VAL = -998.0

def get_memory_usage():
    """获取当前进程内存占用 (MB)"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def generate_massive_data():
    """
    极速生成测试数据 (含噪音注入)
    """
    print(f"🚀 [DataGen] 正在生成 {N_ROWS:,} 行 x {N_COLS} 列数据...")
    t0 = time.time()
    
    # 1. 生成基础数据 (Float32 节省内存)
    data_matrix = np.random.randn(N_ROWS, N_COLS).astype(np.float32)
    
    # 2. ⚡️ 极速注入脏数据 (使用 Numpy Mask 操作)
    # 设定 5% 为缺失值 (-999), 5% 为特殊值 (-998)
    rng = np.random.default_rng(42) # 固定种子保证复现
    mask_missing = rng.random((N_ROWS, N_COLS)) < 0.05
    mask_special = rng.random((N_ROWS, N_COLS)) < 0.05
    
    # 注入特殊值 (先注入，避免覆盖缺失值)
    data_matrix[mask_special] = SPECIAL_VAL
    # 注入缺失值 (优先级更高，覆盖掉可能重叠的特殊值)
    data_matrix[mask_missing] = MISSING_VAL
    
    # 3. 注入 Target (简单的线性关系 + 噪音)
    # 注意：计算 Target 时要剔除脏数据的影响，防止 Target 也变成 NaN
    clean_feature_0 = np.where(data_matrix[:, 0] <= -900, 0, data_matrix[:, 0])
    prob = 1 / (1 + np.exp(-np.clip(clean_feature_0, -10, 10)))
    y = (rng.random(N_ROWS) < prob).astype(int)
    y[mask_missing[:, 0]] = 1
    
    # 4. 转为 Polars
    col_names = [f"f_{i}" for i in range(N_COLS)]
    # 使用 zero-copy 转换
    df = pl.from_numpy(data_matrix, schema=col_names)
    
    print(f"✅ 数据准备完毕: {time.time() - t0:.2f}s | 内存占用: {get_memory_usage():.2f} MB")
    print(f"   - Missing Rate (-999): ~5%")
    print(f"   - Special Rate (-998): ~5%")
    return df, y

def run_stress_test():
    # 1. 生成数据
    df, y = generate_massive_data()
    
    print("\n" + "="*90)
    print(f"🔥 MARS 性能压力测试 (含脏数据处理)")
    print("="*90)
    print(f"{'Method':<15} | {'Fit Time':<15} | {'Transform Time':<15} | {'Mem Overhead':<15}")
    print("-" * 80)

    # 2. 循环测试不同方法
    for method in ["quantile", "uniform", "cart"]:
        gc.collect()
        mem_start = get_memory_usage()
        t0 = time.time()
        
        # --- 初始化与训练 ---
        # 关键点：传入 missing_values 和 special_values
        # 这会触发 _get_safe_values 和 transform 中的复杂过滤逻辑
        binner = MarsNativeBinner(
            method=method, 
            n_bins=10, 
            n_jobs=-1, 
            remove_empty_bins=False, # 大宽表压测建议关闭
            missing_values=[MISSING_VAL, "unknown"], # 故意混入字符串，测试类型安全防御
            special_values=[SPECIAL_VAL]
        )
        
        try:
            binner.fit(df, y)
            t_fit = time.time() - t0
            
            # --- 转换 ---
            t1 = time.time()
            
            # ✅ [Fix] 改回 "index" 模式
            # 目的：验证 dirty data (-999) 是否被正确路由到了 -1 号箱
            res = binner.transform(df, return_type="index", lazy=False)
            
            t_trans = time.time() - t1
            
            # --- 验证 ---
            # 此时 res 中的列是 f_0_bin，值应该是 -1
            sample_chk = res.filter(pl.col("f_0") == MISSING_VAL)
            if sample_chk.height > 0:
                 idx = sample_chk["f_0_bin"][0]
                 status = "✅" if idx == -1 else f"❌(Got {idx})"
            else:
                 status = "⚠️(No Sample)"

            mem_used = get_memory_usage() - mem_start
            print(f"{method:<15} | {t_fit:<15.4f} | {t_trans:<15.4f} | {mem_used:<15.2f} MB {status}")
            
        except Exception as e:
            print(f"{method:<15} | FAILED: {str(e)}")

    # 3. 清理
    print("-" * 80)
    print("🧹 Cleaning up...")
    del df, y
    gc.collect()

if __name__ == "__main__":
    run_stress_test()

🚀 [DataGen] 正在生成 100,000 行 x 5000 列数据...


## **⚔️ 模块 4：竞品对比 (Vs Toad)**
- 说明：与工业界标杆 Toad 进行对比。为了保证 Toad 能跑完，数据量缩小为 20万行 x 50列。

In [ ]:
# ==========================================
# 模块 4: 竞品对比测试 (Vs Toad) - WOE 模式
# ==========================================
import toad
import time
import gc

# 配置
N_ROWS = 200_000      # 20万行
N_COLS = 2000        # 2000特征 (大宽表)
SPECIAL_VAL = -999.0

def benchmark_woe_comparison():
    # 使用 500 列以保证对比显著性
    N_COMP_COLS = 500
    print(f"\n⚔️ [Benchmark] Mars vs Toad WOE Transformation ({N_ROWS}行 x {N_COMP_COLS}列)")
    
    # 准备数据
    data_np = np.random.randn(N_ROWS, N_COMP_COLS).astype(np.float32)
    col_names = [f"f_{i}" for i in range(N_COMP_COLS)]
    y_np = (data_np[:, 0] > 1).astype(int) # 稍微倾斜 target 保证 WOE 有意义
    
    # 转换为各自的容器
    df_pl = pl.from_numpy(data_np, schema=col_names)
    df_pd = pd.DataFrame(data_np, columns=col_names)
    df_pd['target'] = y_np

    results = []

    test_cases = [
        ("cart", "决策树 (CART/DT)"),
        ("quantile", "等频 (Quantile)"),
        ("uniform", "等宽 (Uniform/Step)"),
    ]

    for method, label in test_cases:
        # --- 1. MARS 测试 (WOE 模式) ---
        gc.collect()
        m_base = get_memory_usage()
        
        # Fit 阶段
        t0 = time.time()
        mars_model = MarsNativeBinner(method=method, n_bins=5, n_jobs=-1)
        mars_model.fit(df_pl, y_np)
        t_mars_fit = time.time() - t0
        
        # Transform 阶段 (包含延迟物化 WOE 的时间)
        t1 = time.time()
        # 注意：此处 return_type="woe" 会自动触发 v7.2 的极速批处理物化引擎
        res_pl = mars_model.transform(df_pl, return_type="woe")
        if isinstance(res_pl, pl.LazyFrame):
            res_pl.collect()
        t_mars_trans = time.time() - t1
        m_mars_total = get_memory_usage() - m_base 
        
        # --- 2. TOAD 测试 (WOE 模式) ---
        gc.collect()
        t_base = get_memory_usage()
        toad_method = 'dt' if method == 'cart' else ('step' if method == 'uniform' else method)
        
        try:
            # Fit 阶段 (Combiner 寻找切点)
            t2 = time.time()
            toad_model = toad.transform.Combiner()
            toad_model.fit(df_pd, 'target', method=toad_method, n_bins=5)
            
            
    
            t_toad_fit = time.time() - t2
            
            # Transform 阶段 (实际映射)
            t3 = time.time()
            # TOAD 的 WOE 转换需要两步：1. Combiner 转换索引；2. WOETransformer 转换数值
            # 这里我们将这两步及物化时间全部计入 Transform Time
            combiner_res = toad_model.transform(df_pd)
            woe_trans = toad.transform.WOETransformer()
            # 物化 WOE 映射表
            woe_trans.fit(combiner_res, combiner_res['target'])
            df_toad_res = woe_trans.transform(combiner_res)
            _ = df_toad_res.shape
            t_toad_trans = time.time() - t3
            m_toad_total = get_memory_usage() - t_base
            
            # 计算对比指标
            fit_speedup = t_toad_fit / t_mars_fit
            trans_speedup = t_toad_trans / t_mars_trans
            # 这里的内存对比包含了结果矩阵 (Float64) 的物理体积
            mem_diff = m_toad_total - m_mars_total 
            
        except Exception as e:
            print(f"Toad failed on {method}: {e}")
            t_toad_fit, t_toad_trans, m_toad_total = np.nan, np.nan, np.nan
            fit_speedup, trans_speedup, mem_diff = 0, 0, 0

        results.append({
            "Method": label,
            "M-Fit(s)": t_mars_fit,
            "T-Fit(s)": t_toad_fit,
            "Fit-Spd(x)": fit_speedup,
            "M-Trans+WOE(s)": t_mars_trans, # 包含物化+转换
            "T-Trans+WOE(s)": t_toad_trans,
            "Trans-Spd(x)": trans_speedup,
            "M-TotalMem(MB)": m_mars_total, 
            "T-TotalMem(MB)": m_toad_total,
            "Mem-Gap(MB)": mem_diff # Toad 多用了多少 MB
        })

    # 展示结果
    res_df = pd.DataFrame(results)
    print("\n🏆 MARS vs TOAD WOE 全维度对比:")
    display(res_df.round(3))

benchmark_woe_comparison()

In [ ]:
gc.collect()